In [1]:
import pandas as pd
from IPython.display import display, clear_output

In [2]:
boro_lst = ["Bronx", "Brooklyn", "Queens", "Staten_Island"]

In [3]:
from shapely import wkt
from pyproj import Transformer

# Define the source and target coordinate reference systems (CRS)
# Create a transformer object
SOURCE_CRS: str = "EPSG:2263"  # EPSG using in NewYorkCity
TARGET_CRS: str = "EPSG:4326"  # WGS84 (latitude, longitude)
TRANSFORMER: Transformer = Transformer.from_crs(SOURCE_CRS, TARGET_CRS, always_xy=True)


def convert_wkt_to_coordinates(wkt_str: str) -> tuple[float, float]:
    """
    Converts a WKT string to a tuple of coordinates.

    Args:
        wkt (str): The WKT string.

    Returns:
        tuple[float, float]: The tuple of coordinates.
    """
    wkt_geo = wkt.loads(wkt_str)
    res: tuple[float, float] = TRANSFORMER.transform(wkt_geo.x, wkt_geo.y)  # long, lat
    return res

In [4]:
def add_lat_long(df: pd.DataFrame) -> pd.DataFrame:
    res: pd.DataFrame = df.copy()
    res["tmp"] = res["WktGeom"].apply(convert_wkt_to_coordinates)
    res["Long"] = res["tmp"].apply(lambda x: x[0])
    res["Lat"] = res["tmp"].apply(lambda x: x[1])
    
    res.drop(columns=["tmp"], inplace=True)
    
    return res

In [5]:
def store_df_to_csv(df: pd.DataFrame, **kwags):
    df.to_csv(**kwags)

In [6]:
all_datasets = pd.DataFrame(columns=["RequestID", "Boro", "Yr", "M", "D", "HH", "MM", "Vol", "SegmentID", "WktGeom", "Street", "fromSt", "toSt", "Direction"])

In [7]:
for boro in boro_lst:
    df = pd.read_csv(f"datasets/all/Boro={boro}.csv", header=None)
    df.columns=["RequestID", "Boro", "Yr", "M", "D", "HH", "MM", "Vol", "SegmentID", "WktGeom", "Street", "fromSt", "toSt", "Direction"]
    all_datasets = pd.concat([all_datasets, df], ignore_index=True)

In [8]:
all_datasets = all_datasets[all_datasets["SegmentID"] != 103857].reset_index().drop(columns=["index"])
all_datasets

,RequestID,Boro,Yr,M,D,HH,MM,Vol,SegmentID,WktGeom,Street,fromSt,toSt,Direction
0,26015,Bronx,2017,6,13,0,15,28,86462,POINT (1024865.2440862487 245978.13529199662),CASTLE HILL AVENUE,Zerega Avenue,Bronxdale Avenue,NB
1,26015,Bronx,2017,6,13,0,15,78,85912,POINT (1020499.539565537 250118.4389520207),BRONX PARK EAST,Maran Place,Brady Avenue,SB
2,26015,Bronx,2017,6,13,0,15,6,85918,POINT (1020827.9632961922 249742.51625678336),WHITE PLAINS ROAD,Brady Avenue,Bronxdale Avenue,SB
3,26015,Bronx,2017,6,13,0,15,9,85918,POINT (1020827.9632961922 249742.51625678336),WHITE PLAINS ROAD,Brady Avenue,Bronxdale Avenue,SB
4,26015,Bronx,2017,6,13,0,15,36,85916,POINT (1020821.3217130088 249292.94951331365),WHITE PLAINS ROAD,Birchall Avenue,Bronxdale Avenue,NB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64830,26047,Staten Island,2017,6,21,0,0,8,10855,POINT (954779.467477953 146426.41003341574),NEW DORP LANE,Mill Road,Titus Avenue,WB
64831,26048,Staten Island,2017,6,21,0,0,5,11787,POINT (956840.2147654856 154048.9659754029),JEFFERSON STREET,Seaview Avenue,Liberty Avenue,NB
64832,26048,Staten Island,2017,6,21,0,0,0,11787,POINT (956840.2147654856 154048.9659754029),JEFFERSON STREET,Seaview Avenue,Liberty Avenue,SB
64833,26048,Staten Island,2017,6,21,0,0,8,160346,POINT (958573.6621717077 153329.2799454445),GARRETSON AVENUE,Magnolia Avenue,Hylan Boulevard,EB


In [9]:
handle_df = all_datasets[["Boro", "SegmentID", "WktGeom", "Street", "fromSt", "toSt"]]

In [10]:
handle_df = handle_df.drop_duplicates().reset_index().drop(columns=['index'])
handle_df

,Boro,SegmentID,WktGeom,Street,fromSt,toSt
0,Bronx,86462,POINT (1024865.2440862487 245978.13529199662),CASTLE HILL AVENUE,Zerega Avenue,Bronxdale Avenue
1,Bronx,85912,POINT (1020499.539565537 250118.4389520207),BRONX PARK EAST,Maran Place,Brady Avenue
2,Bronx,85918,POINT (1020827.9632961922 249742.51625678336),WHITE PLAINS ROAD,Brady Avenue,Bronxdale Avenue
3,Bronx,85916,POINT (1020821.3217130088 249292.94951331365),WHITE PLAINS ROAD,Birchall Avenue,Bronxdale Avenue
4,Bronx,86040,POINT (1021090.0094792125 248997.7737956675),CRUGER AVENUE,Dyre Avenue Line,Bronxdale Avenue
...,...,...,...,...,...,...
64,Staten Island,105505,POINT (954501.1489633534 146298.30413088444),MILL ROAD,New Dorp Lane,Windmill Court
65,Staten Island,10855,POINT (954779.467477953 146426.41003341574),NEW DORP LANE,Mill Road,Titus Avenue
66,Staten Island,11787,POINT (956840.2147654856 154048.9659754029),JEFFERSON STREET,Seaview Avenue,Liberty Avenue
67,Staten Island,160346,POINT (958573.6621717077 153329.2799454445),GARRETSON AVENUE,Magnolia Avenue,Hylan Boulevard


In [11]:
handle_df[["SegmentID"]].drop_duplicates().reset_index().drop(columns=['index'])

,SegmentID
0,86462
1,85912
2,85918
3,85916
4,86040
...,...
64,105505
65,10855
66,11787
67,160346


In [12]:
handle_df[["WktGeom"]].drop_duplicates().reset_index().drop(columns=["index"])

,WktGeom
0,POINT (1024865.2440862487 245978.13529199662)
1,POINT (1020499.539565537 250118.4389520207)
2,POINT (1020827.9632961922 249742.51625678336)
3,POINT (1020821.3217130088 249292.94951331365)
4,POINT (1021090.0094792125 248997.7737956675)
...,...
64,POINT (954501.1489633534 146298.30413088444)
65,POINT (954779.467477953 146426.41003341574)
66,POINT (956840.2147654856 154048.9659754029)
67,POINT (958573.6621717077 153329.2799454445)


In [13]:
handle_df[["Boro", "Street", "fromSt", "toSt"]].drop_duplicates().reset_index().drop(columns=["index"])

,Boro,Street,fromSt,toSt
0,Bronx,CASTLE HILL AVENUE,Zerega Avenue,Bronxdale Avenue
1,Bronx,BRONX PARK EAST,Maran Place,Brady Avenue
2,Bronx,WHITE PLAINS ROAD,Brady Avenue,Bronxdale Avenue
3,Bronx,WHITE PLAINS ROAD,Birchall Avenue,Bronxdale Avenue
4,Bronx,CRUGER AVENUE,Dyre Avenue Line,Bronxdale Avenue
...,...,...,...,...
62,Staten Island,MILL ROAD,New Dorp Lane,Windmill Court
63,Staten Island,NEW DORP LANE,Mill Road,Titus Avenue
64,Staten Island,JEFFERSON STREET,Seaview Avenue,Liberty Avenue
65,Staten Island,GARRETSON AVENUE,Magnolia Avenue,Hylan Boulevard


In [14]:
test_df = add_lat_long(handle_df).sort_values(["SegmentID"]).reset_index().drop(columns=["WktGeom", "index"])
test_df = test_df[["SegmentID", "Boro", "Street", "fromSt", "toSt", "Long", "Lat"]]
test_df

,SegmentID,Boro,Street,fromSt,toSt,Long,Lat
0,10855,Staten Island,NEW DORP LANE,Mill Road,Titus Avenue,-74.106075,40.568534
1,11787,Staten Island,JEFFERSON STREET,Seaview Avenue,Liberty Avenue,-74.098688,40.589463
2,39873,Brooklyn,GERRITSEN AVENUE,Bragg Street,Avenue U,-73.935281,40.601956
3,39976,Brooklyn,AVENUE U,Brigham Street,Gerritsen Avenue,-73.935251,40.601590
4,39977,Brooklyn,GERRITSEN AVENUE,Avenue U,Avenue U,-73.934731,40.601464
...,...,...,...,...,...,...,...
64,192771,Queens,HORACE HARDING EXPRESSWAY SR N,260 Street,Dead end,-73.721579,40.764843
65,194239,Bronx,MAJOR DEEGAN EXPRESSWAY SR,Connector,Dead end,-73.878400,40.905772
66,194931,Bronx,BOSTON ROAD,Pelham Parkway,Dead End,-73.869040,40.856992
67,255311,Queens,WHITESTONE EXPRESSWAY ET 14 NB,Dead End,Whitestone Expressway Sr East,-73.837437,40.767607


In [15]:
# test_df.to_csv('datasets/static_files/segment_table.csv', header=True, index=False)
store_df_to_csv(test_df, path_or_buf="datasets/static_files/segment_table.csv", header=True, index=False)

In [16]:
handle_df = all_datasets[["Boro", "SegmentID", "Yr", "M", "D", "HH", "MM", "Direction", "Vol"]]

In [17]:
handle_df["Timestamp"] = pd.to_datetime(handle_df["Yr"].astype("str") + "-" +
                                       handle_df["M"].astype("str") + "-" + 
                                       handle_df["D"].astype("str") + " " + 
                                       handle_df["HH"].astype("str") + ":" + 
                                       handle_df["MM"].astype("str"))

C:\Users\le197\AppData\Local\Temp\ipykernel_17588\3905002019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  handle_df["Timestamp"] = pd.to_datetime(handle_df["Yr"].astype("str") + "-" +


In [18]:
handle_df.drop(columns=["Yr", "M", "D", "HH", "MM"], inplace=True)
handle_df

C:\Users\le197\AppData\Local\Temp\ipykernel_17588\932399640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  handle_df.drop(columns=["Yr", "M", "D", "HH", "MM"], inplace=True)


,Boro,SegmentID,Direction,Vol,Timestamp
0,Bronx,86462,NB,28,2017-06-13 00:15:00
1,Bronx,85912,SB,78,2017-06-13 00:15:00
2,Bronx,85918,SB,6,2017-06-13 00:15:00
3,Bronx,85918,SB,9,2017-06-13 00:15:00
4,Bronx,85916,NB,36,2017-06-13 00:15:00
...,...,...,...,...,...
64830,Staten Island,10855,WB,8,2017-06-21 00:00:00
64831,Staten Island,11787,NB,5,2017-06-21 00:00:00
64832,Staten Island,11787,SB,0,2017-06-21 00:00:00
64833,Staten Island,160346,EB,8,2017-06-21 00:00:00


In [19]:
boro_values = handle_df["Boro"].unique()
static_df = handle_df[handle_df["Timestamp"] <= "2017-06-20"].drop(columns=["Boro"])
store_df_to_csv(static_df, path_or_buf="datasets/static_files/event_table.csv", header=False, index=False)

streaming_df = handle_df[handle_df["Timestamp"] > "2017-06-20"]
for boro_value in boro_values:
    tmp_df = streaming_df[streaming_df["Boro"] == boro_value]
    boro_value = "_".join(boro_value.split(" "))
    print("Boro value: ", boro_value)
    # print(tmp_df.drop(columns=["Boro"]))
    tmp_df.drop(columns=["Boro"], inplace=True)
    
    store_df_to_csv(tmp_df, path_or_buf=f"datasets/streaming_files/Event_table_boro={boro_value}.csv", header=False, index=False)

Boro value:  Bronx
Boro value:  Brooklyn
Boro value:  Queens
Boro value:  Staten_Island


C:\Users\le197\AppData\Local\Temp\ipykernel_17588\3653595582.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df.drop(columns=["Boro"], inplace=True)
C:\Users\le197\AppData\Local\Temp\ipykernel_17588\3653595582.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df.drop(columns=["Boro"], inplace=True)
C:\Users\le197\AppData\Local\Temp\ipykernel_17588\3653595582.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df.dr